In [2]:
# Static variables
BUCKET='elite-caster-125113'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
spark = SparkSession\
  .builder \
  .appName("Lgistic regression w/ Spark ML") \
  .getOrCreate()

In [4]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint

## Creating a Training Dataset

In [6]:
# CSV to Dataframe
traindays: DataFrame = spark.read \
  .option("header", "true") \
  .csv('gs://{}/flights/trainday.csv'.format(BUCKET))

In [7]:
traindays.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- is_train_day: string (nullable = true)



In [8]:
# Register the dataframe as TempView for spark sql
traindays.createOrReplaceTempView('traindays')

In [9]:
spark.sql("SELECT * FROM traindays LIMIT 5").show()

+----------+------------+
|   FL_DATE|is_train_day|
+----------+------------+
|2018-01-02|        True|
|2018-01-03|        True|
|2018-01-04|        True|
|2018-01-05|        True|
|2018-01-07|        True|
+----------+------------+



In [16]:
from pyspark.sql.types import StringType, FloatType, StructType, StructField

header = 'FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,'
header += 'ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,'
header += 'DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,CRS_DEP_TIME,DEP_TIME,'
header += 'DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,'
header += 'CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,'
header += 'CANCELLATION_CODE,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,'
header += 'DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,'
header += 'ARR_AIRPORT_TZOFFSET,EVENT,NOTIFY_TIME'

print(header)

def get_structfield(colname: str) -> StructField:
    if colname in ['ARR_DELAY', 'DEP_DELAY', 'DISTANCE', 'TAXI_OUT']:
        return StructField(colname, FloatType(), True)
    else:
        return StructField(colname, StringType(), True)


schema = StructType([get_structfield(colname) for colname in header.split(',')])

FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,ARR_AIRPORT_TZOFFSET,EVENT,NOTIFY_TIME


In [11]:
inputs = 'gs://{}/flights/tzcorr/flights-00000-*'.format(BUCKET)
# inputs = 'gs://{}/flights/tzcorr/flights-*'.format(BUCKET)

In [15]:
flights: DataFrame = spark.read \
  .schema(schema) \
  .csv(inputs)
    
flights.createOrReplaceTempView('flights')

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: string (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: string (nullable = true)
 |-- DEST_CITY_MARKET_ID: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: float (nullable = true)
 |-- TAXI_OUT: float (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: 

In [13]:
trainquery: str = """
SELECT
  f.*
FROM flights f
JOIN traindays t
ON f.FL_DATE == t.FL_DATE
WHERE
 t.is_train_day == 'True'
"""

traindata: DataFrame = spark.sql(trainquery)

## Dealing with Corner Cases

In [14]:
traindata.head(2)

[Row(FL_DATE='2018-01-02', OP_UNIQUE_CARRIER='9E', OP_CARRIER_AIRLINE_ID='20363', OP_CARRIER='9E', OP_CARRIER_FL_NUM='3615', ORIGIN_AIRPORT_ID='11898', ORIGIN_AIRPORT_SEQ_ID='1189802', ORIGIN_CITY_MARKET_ID='31898', ORIGIN='GFK', DEST_AIRPORT_ID='13487', DEST_AIRPORT_SEQ_ID='1348702', DEST_CITY_MARKET_ID='31650', DEST='MSP', CRS_DEP_TIME='2018-01-02 19:10:00', DEP_TIME='2018-01-02 19:05:00', DEP_DELAY=-5.0, TAXI_OUT=10.0, WHEELS_OFF='2018-01-02 19:15:00', WHEELS_ON='2018-01-02 20:02:00', TAXI_IN='5.00', CRS_ARR_TIME='2018-01-02 20:29:00', ARR_TIME='2018-01-02 20:07:00', ARR_DELAY=-22.0, CANCELLED='0.00', CANCELLATION_CODE=None, DIVERTED='0.00', DISTANCE=284.0, DEP_AIRPORT_LAT='47.94722222', DEP_AIRPORT_LON='-97.17388889', DEP_AIRPORT_TZOFFSET='-21600.0', ARR_AIRPORT_LAT='44.88194444', ARR_AIRPORT_LON='-93.22166667', ARR_AIRPORT_TZOFFSET='-21600.0', EVENT=None, NOTIFY_TIME=None),
 Row(FL_DATE='2018-01-02', OP_UNIQUE_CARRIER='9E', OP_CARRIER_AIRLINE_ID='20363', OP_CARRIER='9E', OP_CARRIE

In [20]:
traindata[["DEP_DELAY", "TAXI_OUT", "ARR_DELAY", "DISTANCE"]].describe().show()

+-------+------------------+------------------+-----------------+-----------------+
|summary|         DEP_DELAY|          TAXI_OUT|        ARR_DELAY|         DISTANCE|
+-------+------------------+------------------+-----------------+-----------------+
|  count|            104732|            104792|           104464|           111534|
|   mean|13.306744834434557|17.099101076418048|6.268293383366519|815.3889576272707|
| stddev| 52.23925364755955| 9.644846467745014|53.40158784227428|605.3003644466102|
|    min|             -49.0|               1.0|            -78.0|             31.0|
|    max|            1752.0|             164.0|           1778.0|           4983.0|
+-------+------------------+------------------+-----------------+-----------------+



In [22]:
# Revise query by putting NULL fields into account
# Flights that were scheduled but 
#   never left the gate (DEP_DELAY is null)
#   never take off (TAXI_OUT is null) 
# Flights took off but diverted and do not have an ARR_DELAY (This includes TAXI_OUT)
trainquery_revised: str = """
SELECT
  DEP_DELAY, TAXI_OUT, ARR_DELAY, DISTANCE
FROM flights f
JOIN traindays t
ON f.FL_DATE == t.FL_DATE
WHERE
 t.is_train_day == 'True' AND
 f.DEP_DELAY IS NOT NULL AND
 f.ARR_DELAY IS NOT NULL
"""
    
traindata: DataFrame = spark.sql(trainquery_revised)
traindata.describe().show()


+-------+------------------+------------------+------------------+-----------------+
|summary|         DEP_DELAY|          TAXI_OUT|         ARR_DELAY|         DISTANCE|
+-------+------------------+------------------+------------------+-----------------+
|  count|            104348|            104348|            104348|           104348|
|   mean|13.169586384022693|17.083997776670373|6.2886686855521905|813.5318261969563|
| stddev| 51.35008338509632|  9.63459230131671|53.425693453588465|602.6741231081046|
|    min|             -49.0|               1.0|             -78.0|             31.0|
|    max|            1752.0|             164.0|            1778.0|           4983.0|
+-------+------------------+------------------+------------------+-----------------+



In [24]:
# I want to fix the root cause instead of fixing the symptom
trainquery_revised_final: str = """
SELECT
  DEP_DELAY, TAXI_OUT, ARR_DELAY, DISTANCE
FROM flights f
JOIN traindays t
ON f.FL_DATE == t.FL_DATE
WHERE
 t.is_train_day == 'True' AND
 f.CANCELLED == '0.00' AND
 f.DIVERTED == '0.00'
"""
    
traindata: DataFrame = spark.sql(trainquery_revised_final)
traindata.describe().show()


+-------+------------------+------------------+-----------------+-----------------+
|summary|         DEP_DELAY|          TAXI_OUT|        ARR_DELAY|         DISTANCE|
+-------+------------------+------------------+-----------------+-----------------+
|  count|            104379|            104497|           104464|           104497|
|   mean|13.166470267007732|17.091227499354048|6.268293383366519|813.0437811611816|
| stddev| 51.34305436913121| 9.639548229308126|53.40158784227428|602.4833101721217|
|    min|             -49.0|               1.0|            -78.0|             31.0|
|    max|            1752.0|             164.0|           1778.0|           4983.0|
+-------+------------------+------------------+-----------------+-----------------+



## Creating Training Examples

In [27]:
# To use Logistic Regression (https://bit.ly/3HGBYpw)
# I first need labled training sets for binary outcomes.
# In this case , positive lable (1) and negative label(0)
def to_example(raw_data_point: DataFrame) -> LabeledPoint:
    return LabeledPoint(\
            float(raw_data_point['ARR_DELAY'] < 15),  # on-time? \
            [ \
                raw_data_point['DEP_DELAY'], \
                raw_data_point['TAXI_OUT'], \
                raw_data_point['DISTANCE'], \
            ])

examples: DataFrame = traindata.rdd.map(to_example)

## Training

In [ ]:
# tbc